# Binary Classification of Subjective/Objective tweets
## Features: 
- POS tags + tfidf count in BoW style

In [2]:
import pandas as p
import numpy as np
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk import pos_tag

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix as cm

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cross_validation import train_test_split

In [3]:
train = p.read_csv('./sub_obj_data/train_new.csv', usecols=(['class', 'text'])).dropna()
train = train.reindex(np.random.permutation(train.index))
train_large = p.read_csv('./sub_obj_data/training.csv', usecols=(['class', 'text'])).dropna()
train_large = train_large.reindex(np.random.permutation(train_large.index))

test  = p.read_csv('./sub_obj_data/test_ds.csv', usecols=(['class', 'text'])).dropna()

print len(train), len(test)

5946 278


# Preprocessor class
Helper class which tokenises tweets and includes pos tags.

In [4]:
from __future__ import unicode_literals, division
import re
import htmlentitydefs
import csv

from nltk.corpus import stopwords as sw
from nltk import pos_tag


class Preprocessor():

    def __init__(self):
        self.stopwords = list(sw.words('english'))
        self.lexicon = self.load_lexicon()
        self.lexicon_feats = {'pos': [], 'neg': [], 'neut': []}
        self.word_re = word_re
        self.emoticon_re = emoticon_re
        self.html_entity_digit_re = html_entity_digit_re
        self.html_entity_alpha_re = html_entity_alpha_re
        self.amp = amp
        self.punct_re = punct_re
        self.negation_re = negation_re
        self.url_re = url_re
        self.rep_char_re = rep_char_re
        self.hashtag_re = hashtag_re
        self.user_tag_re = user_tag_re
        self.acrynoms = self.load_acrynoms()
        self.stemmer = PorterStemmer()

    def load_acrynoms(self):
        with open('./data/acrynom.csv', 'rb') as f:
            reader = csv.reader(f)
            slang = dict((rows[0], rows[1]) for rows in reader)
            return slang
    
    def load_lexicon(self):
        with open('./data/lexicon.csv', 'rb') as f:
            reader = csv.reader(f)
            return dict((rows[2], rows[5]) for rows in reader)
        
    def pos_tags(self, tokens):
        TAG_MAP = [ "NN", "NNP", "NNS", "VBP", "VB", "VBD", 'VBG', "VBN",
                    "VBZ", "MD","UH", "PRP", "PRP$"]
        tags = pos_tag(tokens)
        return [tag[1] for tag in tags if tag[1] in TAG_MAP]

    def reset_feats(self):
        self.lexicon_feats = {k: [] for k, v in self.lexicon_feats.iteritems()}
        
    def normalise_vect(self):
        max_val = 0
        for v in self.lexicon_feats.itervalues():
            temp = max(map(lambda x: x[0], v))
            if temp > max_val:
                max_val = temp

        for k, v in self.lexicon_feats.iteritems():
            self.lexicon_feats[k] = map(lambda x: [0] if x[0] == 0 else [x[0]/max_val], v)
    
    def normalise(self, tokens):
        
        vect = []
        
        for t in tokens:

            if t in string.punctuation or t in self.stopwords:
                continue
                
            if not self.emoticon_re.search(t):
                t = t.lower()
            
            if t in self.acrynoms:
                t = self.acrynoms[t]
            
            t = self.rep_char_re.sub(r'\1', t)
            t = self.url_re.sub('_URL', t)
            t = self.hashtag_re.sub('_HASH', t)
            t = self.user_tag_re.sub('_USER', t)
            
            vect.append(self.stemmer.stem(t))

        tags = self.pos_tags(tokens)
        vect = tags + vect
        return vect
    
    def lexicon_lookup(self, tokens):
        polarities = ['pos', 'neg', 'neut']
        for polarity in polarities:
            self.lexicon_feats[polarity].append([0])
        
        idx = len(self.lexicon_feats['pos']) - 1
        
        for token in tokens:    
            token = token.lower()
            if token in self.lexicon:
                polarity = self.lexicon[token]
                if polarity == '4':
                    self.lexicon_feats['pos'][idx][0] += 1
                elif polarity == '0':
                    self.lexicon_feats['neg'][idx][0] += 1
                elif polarity == '2': 
                    self.lexicon_feats['neut'][idx][0] += 1
#             else:
#                 print 'not in lexicon'
                
#     def append_lexicon_feats(self, present, polarity):
#         polarities = ['pos', 'neg', 'neut']
#         for polarity in polarities:
#             self.feats[polarity].append([0])
        
#         idx = len(self.feats['pos']) - 1
#         if present:
#             self.feats[polarity].append([1])
#         else:
#             self.feats[polarity].append([0])
    
    def tokenise(self, tweet):
        tweet = self.__html2unicode(tweet)
        tokens = self.word_re.findall(tweet)
#         self.lexicon_lookup(tokens)
        return self.normalise(tokens)
    
    def tokenise_raw(self, tweet):
        tweet = self.__html2unicode(tweet)
        tokens = self.word_re.findall(tweet)

    def ensure_unicode(self, tweet):
        try:
            return unicode(tweet)
        except UnicodeDecodeError:
            tweet = str(tweet).encode('string_escape')
            return unicode(tweet)

    def __html2unicode(self, s):
        """
        This function is curtosy of Christopher Potts
        http://sentiment.christopherpotts.net/index.html
        Internal metod that seeks to replace all the HTML entities in
        s with their corresponding unicode characters.
        """
        # First the digits:
        ents = set(self.html_entity_digit_re.findall(s))
        if len(ents) > 0:
            for ent in ents:
                entnum = ent[2:-1]
                try:
                    entnum = int(entnum)
                    s = s.replace(ent, unichr(entnum))
                except:
                    pass
        # Now the alpha versions:
        ents = set(self.html_entity_alpha_re.findall(s))
        ents = filter((lambda x: x != amp), ents)
        for ent in ents:
            entname = ent[1:-1]
            try:
                s = s.replace(ent,
                              unichr(htmlentitydefs.name2codepoint[entname]))
            except:
                pass
            s = s.replace(self.amp, " and ")
        return s

    
"""
    This file is based on the work of Christopher Potts
    however the file has been altered and extended for
    my purposes
    http://sentiment.christopherpotts.net/index.html
"""
emoticon_string = r"""
    (?:
      [<>]?
      [:;=8]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8]                     # eyes
      [<>]?
    )"""

# The components of the tokenizer:
regex_strings = (
    # Phone numbers:
    r""""
    (?:
      (?:            # (international)
        \+?[01]
        [\-\s.]*
      )?
      (?:            # (area code)
        [\(]?
        \d{3}
        [\-\s.\)]*
      )?
      \d{3}          # exchange
      [\-\s.]*
      \d{4}          # base
    )""",
    # Emoticons:
    emoticon_string,
    # HTML tags:
    r'<[^>]+>',
    # Twitter username:
    r'(?:@[\w_]+)',
    # Links
    r'http\S+',
    # Twitter hashtags:
    r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)',
    # Remaining word types:
    r"""
    (?:[a-z][a-z'\-_]+[a-z])       # Words with apostrophes or dashes.
    |
    (?:[+\-]?\d+[,/.:-]\d+[+\-]?)  # Numbers, including fractions, decimals.
    |
    (?:[\w_]+)                    # Words without apostrophes or dashes.
    |
    (?:\.(?:\s*\.){1,})            # Ellipsis dots.
    |
    (?:\S)                         # Everything else that isn't whitespace
    """
    )

negation_words = (
    """
    (?x)(?:
    ^(?:never|no|nothing|nowhere|noone|none|not|
        havent|hasnt|hadnt|cant|couldnt|shouldnt|
        wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint
     )$
    )
    |
    n't
    """
    )

# ######################################################################

word_re = re.compile(r'(%s)' % "|".join(regex_strings), re.VERBOSE | re.I | re.UNICODE)
emoticon_re = re.compile(regex_strings[1], re.VERBOSE | re.I | re.UNICODE)
html_entity_digit_re = re.compile(r'&#\d+;')
html_entity_alpha_re = re.compile(r'&\w+;')
amp = "&amp;"
punct_re = re.compile("^[.:;!?]$")
negation_re = re.compile(negation_words)
url_re = re.compile(r'http\S+')
rep_char_re = re.compile(r'(\w)\1{3,}')
hashtag_re = re.compile(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)')
user_tag_re = re.compile(r'(?:@[\w_]+)')

# Train and evaluate models

In [5]:
import scipy.sparse as sp

import numpy as np


class FeatureCombiner(object):

    def transform(self, X, pre):
        pre.normalise_vect()
        feats = X
        for k, v in pre.lexicon_feats.iteritems():
            feats = np.c_[feats, np.array(v)]
        return feats

    def fit(self, X, y=None):
        return self


def build_and_evaluate(X, y, X_test, y_test, n_gram, min_df, max_df, norm, clf, outpath=None):

    def preprocess(s):
        return preprocessor.tokenise(s)

    # Initialise transformers/estimators
    preprocessor = Preprocessor()
    feat_comb = FeatureCombiner()
    vec = TfidfVectorizer(tokenizer=preprocess,
                          lowercase=False,
                          ngram_range=n_gram,
                          min_df=min_df,
                          max_df=max_df, 
                          norm=norm)
#                           max_features=5000)
    

    # Build model
    print("Building model")
    tfidf_matrix = vec.fit_transform(X)
#     feat_matrix = feat_comb.transform(tfidf_matrix.todense(),
#                                       preprocessor)
    clf.fit(tfidf_matrix, y)

    # Evaluate on test set
#     preprocessor.reset_feats()
    tfidf_matrix_test = vec.transform(X_test)
#     feat_matrix_test = feat_comb.transform(tfidf_matrix_test.todense(),
#                                       preprocessor)
    y_pred = clf.predict(tfidf_matrix_test)

    print("Classification Report:\n")
    print np.mean(y_pred == y_test)
    print cm(y_test, y_pred)
    print(clsr(y_test, y_pred, target_names=['obj', 'sub']))

# Lexicon features

In [44]:
# lexicon = p.read_csv('./data/lexicon.csv').dropna()
# neg = lexicon[lexicon['polarity'] == 0].count()
# neu = lexicon[lexicon['polarity'] == 2].count()
# pos = lexicon[lexicon['polarity'] == 4].count()

# def lexicon_lookup(tokens):
    
#     for token in tokens:    
#         contains = lexicon['polarity'].loc[lexicon['word'] == token]
#         if contains.values == [4]:
#             print 'pos'
#         elif contains.values == [0]:
#             print 'neg'
#         elif contains.values == [2]: 
#             print 'neut'
#         else:
#             print 'not in lexicon'
            

tokenisor = Preprocessor()
# tokenisor.lexicon
# print tokenisor.lexicon['happy']
# if 'happy' in tokenisor.lexicon:
#     polarity = tokenisor.lexicon['happy']
#     print polarity
# lexicon_lookup(['happy'])
tokenisor.lexicon_lookup(['happy', 'happy', 'abandon', 'table', 'seperate', 'excited'])
print tokenisor.lexicon_feats
# tokenisor.lexicon_lookup(['happy', 'happy', 'abandon', 'table', 'seperate', 'excited'])
# print tokenisor.lexicon_feats

{u'neut': [0], u'neg': [1], u'pos': [3]}


# Logistical Regression classifier

In [57]:
clf = LogisticRegression()
n_gram=(1, 1)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.636690647482
[[87 52]
 [49 90]]
             precision    recall  f1-score   support

        obj       0.64      0.63      0.63       139
        sub       0.63      0.65      0.64       139

avg / total       0.64      0.64      0.64       278



# Naive Bayes

In [6]:
clf = MultinomialNB()
n_gram = (1, 1)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram , 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.687050359712
[[ 86  53]
 [ 34 105]]
             precision    recall  f1-score   support

        obj       0.72      0.62      0.66       139
        sub       0.66      0.76      0.71       139

avg / total       0.69      0.69      0.69       278



# Linear SVC classifier

In [54]:
clf = LinearSVC(C=5)#, penalty='l1', dual=False)
n_gram=(1, 2)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.568345323741
[[92 47]
 [73 66]]
             precision    recall  f1-score   support

        obj       0.56      0.66      0.61       139
        sub       0.58      0.47      0.52       139

avg / total       0.57      0.57      0.56       278



# SGD Classifier

In [55]:
clf = SGDClassifier()
n_gram=(1, 2)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.589928057554
[[103  36]
 [ 78  61]]
             precision    recall  f1-score   support

        obj       0.57      0.74      0.64       139
        sub       0.63      0.44      0.52       139

avg / total       0.60      0.59      0.58       278



# Logistical Regression classifier large training set

In [ ]:
clf = LogisticRegression(C=7)
n_gram=(1, 2)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

# Naive Bayes large training set

In [56]:
clf = MultinomialNB()
n_gram = (1, 1)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram , 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.615107913669
[[ 54  85]
 [ 22 117]]
             precision    recall  f1-score   support

        obj       0.71      0.39      0.50       139
        sub       0.58      0.84      0.69       139

avg / total       0.64      0.62      0.59       278



# Linear SVC classifier large training set

In [ ]:
clf = LinearSVC(C=5)#, penalty='l1', dual=False)
n_gram=(1, 2)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

# SGD Classifier large training set

In [ ]:
clf = SGDClassifier()
n_gram=(1, 2)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)